In [1]:
#### Dependencies
import os
import urllib
from urllib.request import urlopen
import json
import csv
import simplejson
import pandas as pd
import pytest
import base64
import sys
import glob
import time
import progressbar

bar = progressbar.ProgressBar()
for i in bar(range(100)):
    time.sleep(0.02)
    
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from bs4 import BeautifulSoup
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

100% (100 of 100) |#######################| Elapsed Time: 0:00:02 Time: 0:00:02


In [ ]:
#Web Scrapping
quote_page = "https://www.pro-football-reference.com/super-bowl/"

#Query the website to get HTML page of the url
html_page = urlopen(quote_page)
html_page
#Parse the html using beautiful soup and store in a variable
sb_parser = BeautifulSoup(html_page, "html.parser")
superbowl_id = sb_parser.find(id = "all_super_bowls")
super_bowl_scores = superbowl_id.text.strip() # strip() is used to remove starting and trailing
# super_bowl_scores.to_json("Super_Bowl_Scores")
super_bowl_dumps = json.dumps(super_bowl_scores)
super_bowl_json = json.loads(super_bowl_dumps)
super_bowl_json.replace("\n", " ")
# super_bowl_json.to_json("Super_Bowl_Scores")
# print(super_bowl_json)
sb_DF = StringIO(super_bowl_json)
super_bowl_DF = sb_DF.writable()
#Using Pandas to webscrape
html_df, = pd.read_html(quote_page)
#Store as JSON and CSV
superbowl_DF = html_df.to_csv("Super_Bowl_Scores.csv", sep=",")

In [ ]:
# Open CSV

sb_output = "Super_Bowl_Scores.csv"
# sb_open = open(sb_output, "r")
# sb_open_csv = sb_open.read()
# sb_open_csv
sb_open = pd.read_csv(sb_output)
new_col = ["Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]
del sb_open ["Unnamed: 0"]
sb_open.head()
# sb_open.set_index("SB")
# sb_csv_print = json.loads(sb_open_csv)
# sb_csv_print

In [ ]:
#Open JSON
sb_output = "Super_Bowl_Scores.json"
sb_open = open(sb_output, "r")
sb_open_json = sb_open.read()
sb_json_print = json.loads(sb_open_json)
pprint(sb_json_print)
sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty.keys())
#Came out much worse than pprint
# e = json.dumps(d, separators=(" ", ":"), sort_keys=True)
# e.replace(":", " ")
# # e.replace("\n", " ")

In [ ]:
# SB_DF.from_csv("Super_Bowl_Scores.csv")

In [ ]:
#SuperBowl DataFrame
sb_dates = []
sb_number = []
sb_mvp = []
sb_winner = []
sb_winPts = []
sb_loser = []
sb_losePts = []
sb_stadium = []
sb_city = []
sb_state = []

for data in sb_json_print:
    sb_dates.append(sb_json_print["Date"])
    sb_number.append(sb_json_print["SB"])
    sb_mvp.append(sb_json_print["MVP"])
    sb_winner.append(sb_json_print["Winner"])
    sb_winPts.append(sb_json_print["Pts"])
    sb_loser.append(sb_json_print["Loser"])
    sb_losePts.append(sb_json_print["Pts.1"])
    sb_stadium.append(sb_json_print["Stadium"])
    sb_city.append(sb_json_print["City"])
    sb_state.append(sb_json_print["State"])
    SB_DF = pd.DataFrame.from_dict(sb_json_print)
SB_DF = SB_DF[["SB", "Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]]
SB_dataDF = SB_DF.rename(index=str, columns={"Pts": "Winning Score", "Pts.1": "Losing Score"})
SB_dataDF
# sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty)


In [ ]:
NFL_data = "box_scores.csv"
NFL_data = pd.read_csv(NFL_data)
NFL_data.head(5)

In [ ]:
team_scores = NFL_data[["date","visitor","home","visitor_score","home_score"]]
team_scores.head(5)

In [ ]:
team_scores.nlargest(5, "home_score")

In [ ]:
team_scores.nlargest(5, "visitor_score")

In [ ]:
NFL_games = "nfl_games.csv"
NFL_games = pd.read_csv(NFL_games)
NFL_date_games = NFL_games.set_index("date")

NFL_date_games = NFL_date_games["2014-09-04":]

In [ ]:
NFL_date_games = NFL_date_games[["season","team1","team2","score1","score2"]]
nfl_scores = NFL_date_games
nfl_scores.head()

In [ ]:
team1_scores = nfl_scores.groupby("team1").sum()["score1"]
team2_scores = nfl_scores.groupby("team2").sum()["score2"]

team1_scores_df = pd.DataFrame(team1_scores)
team2_scores_df = pd.DataFrame(team2_scores)

In [ ]:
#Scores from 2014 to 2017
nfl_team_scores = pd.concat([team1_scores_df, team2_scores_df], axis=1)
nfl_team_scores["Total Points from 2014 to 2017"] = nfl_team_scores["score1"] + nfl_team_scores["score2"]
del nfl_team_scores["score1"]
del nfl_team_scores["score2"]
nfl_team_scores.nlargest(10, "Total Points from 2014 to 2017")

In [ ]:
# Obtained all scores for season, including full season, called DataFrames on all.

nfl_season_2014 = NFL_date_games["2014-09-04":"2015-02-01"]
scores_2014_team1 = nfl_season_2014.groupby("team1").sum()["score1"]
scores_2014_team2 = nfl_season_2014.groupby("team2").sum()["score2"]
scores_2014_df1 = pd.DataFrame(scores_2014_team1)
scores_2014_df2 = pd.DataFrame(scores_2014_team2)
scores_2014 = pd.concat([scores_2014_df1, scores_2014_df2], axis = 1)
scores_2014["Total Points for 2014"] = scores_2014["score1"] + scores_2014["score2"]
del scores_2014["score1"]
del scores_2014["score2"]

nfl_season_2015 = NFL_date_games["2015-09-10":"2016-02-07"]
scores_2015_team1 = nfl_season_2015.groupby("team1").sum()["score1"]
scores_2015_team2 = nfl_season_2015.groupby("team2").sum()["score2"]
scores_2015_df1 = pd.DataFrame(scores_2015_team1)
scores_2015_df2 = pd.DataFrame(scores_2015_team2)
scores_2015 = pd.concat([scores_2015_df1, scores_2015_df2], axis = 1)
scores_2015["Total Points for 2015"] = scores_2015["score1"] + scores_2015["score2"]
del scores_2015["score1"]
del scores_2015["score2"]


nfl_season_2016 = NFL_date_games["2016-09-08":"2017-02-05"]
scores_2016_team1 = nfl_season_2016.groupby("team1").sum()["score1"]
scores_2016_team2 = nfl_season_2016.groupby("team2").sum()["score2"]
scores_2016_df1 = pd.DataFrame(scores_2016_team1)
scores_2016_df2 = pd.DataFrame(scores_2016_team2)
scores_2016 = pd.concat([scores_2016_df1, scores_2016_df2], axis = 1)
scores_2016["Total Points for 2016"] = scores_2016["score1"] + scores_2016["score2"]
del scores_2016["score1"]
del scores_2016["score2"]


nfl_team_scores
scores_2014
scores_2015
scores_2016

In [ ]:
#Getting final row for sums for each DF

nfl_team_scores_sum = nfl_team_scores.loc['Total', 'Sum of Point']= nfl_team_scores['Total Points from 2014 to 2017'].sum()
scores_2014_sum = scores_2014.loc['Total', 'Sum of Point']= scores_2014['Total Points for 2014'].sum()
scores_2015_sum = scores_2015.loc['Total', 'Sum of Point']= scores_2015['Total Points for 2015'].sum()
scores_2016_sum = scores_2016.loc['Total', 'Sum of Point']= scores_2016['Total Points for 2016'].sum()

In [ ]:
sum_all = nfl_team_scores["Total Points from 2014 to 2017"].sum()
sum_all

In [ ]:
score_DF = pd.DataFrame(nfl_scores[['season', 'team1', 'team2', 'score1', 'score2']])
nfl_score_DF = score_DF.append({"Wins": "win", "Losses": "loss"}, ignore_index=True)
final_score_df = nfl_score_DF.fillna(0)
final_score_df.head()

In [ ]:
#DataFrame on Wins/Losses
### 2013-2017 All Wins DF

loss = []
win = []
count = 1
# for scores in final_score_df:
#     if "score1" > "score2":
#         final_score_df["Wins"].append(count + 1)
# final_score_df
for scores in final_score_df:
    if scores in final_score_df["score1"] > scores in final_score_df["score2"]:
        final_score_df["Wins"].append(count + 1)
    elif scores in final_score_df["score2"] > scores in final_score_df["score1"]:
        "Losses".append(count + 1)
        count = count + 1
final_score_df.head()

# Cumulative Player Stats
### Will Be Merged to Analyze Team Stats

In [ ]:
nfl_stats = "Cumulative_Stats_Combined.csv"
nfl_Stats = pd.read_csv(nfl_stats)
nfl_Stats.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
nfl_Stats.set_index("Season")
# nfl_Stats.to_csv("NFL_Cumulative_Stats_Combined.csv")

#  Building Team Stats

In [ ]:
#Sort Stat Leaders by Team
nfl_Stats.groupby(nfl_Stats.index).sum()
teamTotals = nfl_Stats.reset_index().groupby("#Team Abbr.").sum()
teamTotals.keys()

##  Offensive Categories

In [ ]:
#Offensive Totals
offTotals = teamTotals[["#PassAttempts", "#PassCompletions", "#PassPct", "#PassAttemptsPerGame", 
                        "#PassYards", "#PassYardsPerAtt", "#PassYardsPerGame", "#PassTD", "#PassTDPct", 
                        "#PassInt", "#PassIntPct", "#PassLng", "#PassAvg", "#Pass20Plus", 
                        "#Pass40Plus", "#PassSacks", "#PassSackY", "#QBRating", "#RushAttempts", 
                        "#RushAttemptsPerGame", "#RushYards", "#RushAverage", "#RushYardsPerGame", "#RushTD", 
                        "#RushLng", "#Rush1stDowns", "#Rush1stDownsPct", "#Rush20Plus", "#Rush40Plus", 
                        "#RushFumbles", "#Targets", "#Receptions", "#RecYards", "#RecAverage", 
                        "#RecYardsPerGame", "#RecTD", "#RecLng", "#Rec1stDowns", "#Rec20Plus", 
                        "#Rec40Plus", "#RecFumbles"
                       ]]
offTotals.head()

In [ ]:
#Passing Offense
passOff = offTotals[["#PassAttempts", "#PassCompletions", "#PassPct", "#PassAttemptsPerGame", 
                     "#PassYards", "#PassYardsPerAtt", "#PassYardsPerGame", "#PassTD", "#PassTDPct", 
                     "#PassInt", "#PassIntPct", "#PassLng", "#PassAvg", "#Pass20Plus", 
                     "#Pass40Plus", "#PassSacks", "#PassSackY", "#QBRating"]]
passOff.head()

In [ ]:
#Rushing Offense
rushOff = offTotals[["#RushAttemptsPerGame", "#RushYards", "#RushAverage", "#RushYardsPerGame", "#RushTD", 
                     "#RushLng", "#Rush1stDowns", "#Rush1stDownsPct", "#Rush20Plus", "#Rush40Plus", 
                     "#RushFumbles"]]
rushOff.head()

In [ ]:
#Receiving Offense
recOff = offTotals[["#Targets", "#Receptions", "#RecYards", "#RecAverage", 
                    "#RecYardsPerGame", "#RecTD", "#RecLng", "#Rec1stDowns", "#Rec20Plus", 
                    "#Rec40Plus", "#RecFumbles"]]
recOff.head()

In [ ]:
offScores = offTotals[["#PassTD", "#RushTD", "#RecTD"]]
offScores

In [ ]:
offTurn_Sack = offTotals[["#PassInt", "#PassIntPct", "#PassSacks", "#PassSackY", "#RushFumbles", "#RecFumbles"]]
offTurn_Sack.head()

## Defensive Categories

In [ ]:
defTotals = teamTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds", 
                        "#TacklesForLoss", "#Interceptions", "#IntTD", "#IntYds", "#IntAverage", 
                        "#IntLng", "#PassesDefended", "#Stuffs", "#Safeties", "#StuffYds",
                        "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                        "#FumRecYds", "#FumTotalRec", "#FumTD"
]]
defTotals.head()

In [ ]:
passDef = defTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds",
                     "#Interceptions", "#IntTD", "#IntYds", "#IntAverage", "#IntLng", 
                     "#PassesDefended", "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", 
                     "#FumOppRec", "#FumRecYds", "#FumTotalRec", "#FumTD", "#Safeties"]]
passDef.head()

In [ ]:
rushDef = defTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds", 
                     "#TacklesForLoss", "#Stuffs", "#Safeties", "#StuffYds", "#Fumbles", 
                     "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                     "#FumRecYds", "#FumTotalRec", "#FumTD"]]
rushDef.head()

In [ ]:
turnDef = defTotals[["#Interceptions", "#IntTD", "#IntYds", "#IntAverage", "#IntLng",
                     "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                     "#FumRecYds", "#FumTotalRec", "#FumTD"]]
turnDef.head()

In [ ]:
defScores = defTotals[["#IntTD", "#FumTD"]]
defScores

## Special Teams Categories

In [ ]:
sptTotals = teamTotals[["#KB", "#KrRet", "#KrYds", "#KrAvg", "#KrLng","#KrTD", 
                        "#Kr20Plus", "#Kr40Plus", "#KrFC", "#KrFum", "#PrRet", 
                        "#PrYds", "#PrAvg", "#PrLng", "#PrTD", "#Pr20Plus", 
                        "#Pr40Plus", "#PrFC", "#PrFum", "#FgBlk", "#FgMade", 
                        "#FgAtt","#FgPct", "#FgMade1_19", "#FgAtt1_19", "#Fg1_19Pct",
                        "#FgMade20_29", "#FgAtt20_29", "#Fg20_29Pct", "#FgMade30_39", "#FgAtt30_39",
                        "#Fg30_39Pct","#FgMade40_49", "#FgAtt40_49", "#Fg40_49Pct", "#FgMade50Plus", "#FgAtt50Plus",
                        "#Fg50PlusPct", "#FgLng", "#XpBlk", "#XpMade", "#XpAtt", 
                        "#XpPct", "#FgAndXpPts", "#KoPct", "#Kickoffs", "#KoYds",
                        "#KoOOB", '#KoAvg', "#KoTB", "#KoRet", "#KoRetYds", 
                        "#KoRetAvgYds", "#KoTD", "#KoOS", "#KoOSR", "#Punts", 
                        "#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                        "#PuntBlk", "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", 
                        "#PuntTB", "#PuntTBPct", "#PuntFC", "#PuntRet", "#PuntRetYds", "#PuntRetAvg"
]]
sptTotals.head()

In [ ]:
puntRETtotals = teamTotals[["#PrRet", 
                        "#PrYds", "#PrAvg", "#PrLng", "#PrTD", "#Pr20Plus", 
                        "#Pr40Plus", "#PrFC", "#PrFum","#PuntRet", "#PuntRetYds", "#PuntRetAvg"]]
puntRETtotals

In [ ]:
puntTotals = teamTotals[["#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                        "#PuntBlk", "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", 
                        "#PuntTB", "#PuntTBPct", "#PuntFC"]]
puntTotals

In [ ]:
krTotals = teamTotals[["#KrRet", "#KrYds", "#KrAvg", "#KrLng","#KrTD", 
                        "#Kr20Plus", "#Kr40Plus", "#KrFC", "#KrFum","#KoRet", "#KoRetYds", 
                        "#KoRetAvgYds", "#KoTD", "#KoOS", "#KoOSR"]]
krTotals.head()

In [ ]:
kickOFFtotals = teamTotals[["#KoPct", "#Kickoffs", "#KoYds",
                        "#KoOOB", '#KoAvg', "#KoTB"]]
kickOFFtotals.head()

In [ ]:
blockTotals = teamTotals[["#KB", "#FgBlk", "#XpBlk", "#PuntBlk"]]
blockTotals.head()

In [ ]:
fg_xpTotals = teamTotals[["#FgMade", "#FgAtt","#FgPct", "#FgMade1_19", "#FgAtt1_19", "#Fg1_19Pct",
                          "#FgMade20_29", "#FgAtt20_29", "#Fg20_29Pct", "#FgMade30_39", "#FgAtt30_39",
                          "#Fg30_39Pct","#FgMade40_49", "#FgAtt40_49", "#Fg40_49Pct", "#FgMade50Plus", 
                          "#FgAtt50Plus", "#Fg50PlusPct", "#FgLng", "#XpMade", "#XpAtt", 
                          "#XpPct", "#FgAndXpPts",]]
fg_xpTotals.head()

In [ ]:
puntTotals = teamTotals[["#Punts", "#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                         "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", "#PuntTB", 
                         "#PuntTBPct", "#PuntFC", "#PuntRet", "#PuntRetYds", "#PuntRetAvg"]]
puntTotals.head()

In [ ]:
sptTurn = teamTotals[["#KB", "#KrFum", "#PrFum", "#FgBlk", "#XpBlk", "#PuntBlk"]]
sptTurn.head()

In [ ]:
sptScores = teamTotals[["#KrTD", "#PrTD", "#FgMade", "#XpMade", "#FgAndXpPts", "#KoTD"]]
sptScores.head()

## NFL Standings

In [71]:
#Appending Season Column before combining

ov2014 = "NFL_Standings/Division/2014REGULAR_Division.csv"
ov2014 = pd.read_csv(ov2014)
ov2014.insert(1, "Season", "2014-2015-Regular")
ov2014.to_csv("NFL_Standings/Division/2014REGULAR_Division.csv")

ov2014P = "NFL_Standings/Division/2015PLAYOFF_Division.csv"
ov2014P = pd.read_csv(ov2014P)
ov2014P.insert(1, "Season", "2014-2015-Playoff")
ov2014P.to_csv("NFL_Standings/Division/2015PLAYOFF_Division.csv")

ov2015 = "NFL_Standings/Division/2015REGULAR_Division.csv"
ov2015 = pd.read_csv(ov2015)
ov2015.insert(1, "Season", "2015-2016-Regular")
ov2015.to_csv("NFL_Standings/Division/2015REGULAR_Division.csv")

ov2015P = "NFL_Standings/Division/2016PLAYOFF_Division.csv"
ov2015P = pd.read_csv(ov2015P)
ov2015P.insert(1, "Season", "2015-2016-Playoff")
ov2015P.to_csv("NFL_Standings/Division/2016PLAYOFF_Division.csv")

ov2016 = "NFL_Standings/Division/2016REGULAR_Division.csv"
ov2016 = pd.read_csv(ov2016)
ov2016.insert(1, "Season", "2016-2017-Regular")
ov2016.to_csv("NFL_Standings/Division/2016REGULAR_Division.csv")

ov2016P = "NFL_Standings/Division/2017PLAYOFF_Division.csv"
ov2016P = pd.read_csv(ov2016P)
ov2016P.insert(1, "Season", "2016-2017-Playoff")
ov2016P.to_csv("NFL_Standings/Division/2017PLAYOFF_Division.csv")

ov2017 = "NFL_Standings/Division/2017REGULAR_Division.csv"
ov2017 = pd.read_csv(ov2017)
ov2017.insert(1, "Season", "2017-2018-Regular")
ov2017.to_csv("NFL_Standings/Division/2017REGULAR_Division.csv")

ov2017P = "NFL_Standings/Division/2018PLAYOFF_Division.csv"
ov2017P = pd.read_csv(ov2017P)
ov2017P.insert(1, "Season", "2017-2018-Playoff")
ov2017P.to_csv("NFL_Standings/Division/2018PLAYOFF_Division.csv")

In [72]:
#Merge 8 CSV into 1 combined
conference = glob.glob("NFL_Standings/Division/*.csv")
#Loop to Combine ALl DF
conference_DF = []
for filename in sorted(conference):
    conference_DF.append(pd.read_csv(filename))
confStandings_DF = pd.concat(conference_DF, join="inner")

confStandings_DF.to_csv("NFL_Standings/Division/Division_Standings_Combined.csv")
confStandings_DF

,Unnamed: 0,Season,#Division Name,#Team ID,#Team Abbr.,#Team City,#Team Name,#Rank,#GamesPlayed,#PassAttempts,...,#TotalTD,#Wins,#Losses,#Ties,#OTWins,#OTLosses,#WinPct,#PointsFor,#PointsAgainst,#PointDifferential
0,0,2014-2015-Regular,AFC/AFC East,50,NE,New England,Patriots,1,16,627,...,51,12,4,0,0,0,0.000,461,313,148
1,1,2014-2015-Regular,AFC/AFC East,48,BUF,Buffalo,Bills,2,16,599,...,32,9,7,0,0,0,0.000,331,285,46
2,2,2014-2015-Regular,AFC/AFC East,49,MIA,Miami,Dolphins,3,16,601,...,40,8,8,0,0,0,0.000,384,373,11
3,3,2014-2015-Regular,AFC/AFC East,51,NYJ,New York,Jets,4,16,498,...,27,4,12,0,0,1,0.000,283,399,-116
4,4,2014-2015-Regular,AFC/AFC North,57,CIN,Cincinnati,Bengals,1,16,501,...,39,10,5,0,0,0,0.000,359,346,13
5,5,2014-2015-Regular,AFC/AFC North,59,PIT,Pittsburgh,Steelers,2,16,620,...,44,11,5,0,0,0,0.000,428,368,60
6,6,2014-2015-Regular,AFC/AFC North,56,BAL,Baltimore,Ravens,3,16,559,...,44,10,6,0,0,0,0.000,403,302,101
7,7,2014-2015-Regular,AFC/AFC North,58,CLE,Cleveland,Browns,4,16,512,...,30,7,9,0,0,0,0.000,298,337,-39
8,8,2014-2015-Regular,AFC/AFC South,65,IND,Indianapolis,Colts,1,16,669,...,52,11,5,0,0,0,0.000,458,367,91
9,9,2014-2015-Regular,AFC/AFC South,64,HOU,Houston,Texans,2,16,485,...,36,9,7,0,0,0,0.000,374,305,69


### Overall (Each Season & Total)

In [76]:
overall = "NFL_Standings/Overall/Overall_Standings_Combined.csv"
overall = pd.read_csv(overall)
overall.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
overall.set_index("Season")

,#Team ID,#Team Abbr.,#Team City,#Team Name,#Rank,#GamesPlayed,#PassAttempts,#PassCompletions,#PassPct,#PassAttemptsPerGame,...,#TotalTD,#Wins,#Losses,#Ties,#OTWins,#OTLosses,#WinPct,#PointsFor,#PointsAgainst,#PointDifferential
Season,,,,,,,,,,,,,,,,,,,,,
2014-2015-Regular,52,DAL,Dallas,Cowboys,1,16,473,328,69.3,29.6,...,54,12,4,0,0,1,0.000,467,350,117
2014-2015-Regular,72,DEN,Denver,Broncos,2,16,609,399,65.5,38.1,...,56,12,4,0,0,0,0.000,480,349,131
2014-2015-Regular,62,GB,Green Bay,Packers,3,16,551,349,63.3,34.4,...,53,12,4,0,0,0,0.000,475,346,129
2014-2015-Regular,79,SEA,Seattle,Seahawks,4,16,459,287,62.5,28.7,...,41,12,4,0,0,0,0.000,396,252,144
2014-2015-Regular,50,NE,New England,Patriots,5,16,627,392,62.5,39.2,...,51,12,4,0,0,0,0.000,461,313,148
2014-2015-Regular,61,DET,Detroit,Lions,6,16,605,365,60.3,37.8,...,34,11,5,0,0,0,0.000,319,282,37
2014-2015-Regular,57,CIN,Cincinnati,Bengals,7,16,501,321,64.1,31.3,...,39,10,5,0,0,0,0.000,359,346,13
2014-2015-Regular,59,PIT,Pittsburgh,Steelers,8,16,620,411,66.3,38.8,...,44,11,5,0,0,0,0.000,428,368,60
2014-2015-Regular,76,ARI,Arizona,Cardinals,9,16,557,310,55.7,34.8,...,28,11,5,0,0,0,0.000,304,299,5


### Conference (Each Season & Total)

In [78]:
conference = "NFL_Standings/Conference/Conference_Standings_Combined.csv"
conference = pd.read_csv(conference)
conference.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
conference.set_index("Season")

,#Conference Name,#Team ID,#Team Abbr.,#Team City,#Team Name,#Rank,#GamesPlayed,#PassAttempts,#PassCompletions,#PassPct,...,#TotalTD,#Wins,#Losses,#Ties,#OTWins,#OTLosses,#WinPct,#PointsFor,#PointsAgainst,#PointDifferential
Season,,,,,,,,,,,,,,,,,,,,,
2014-2015-Regular,AFC,50,NE,New England,Patriots,1,16,627,392,62.5,...,51,12,4,0,0,0,0.000,461,313,148
2014-2015-Regular,AFC,72,DEN,Denver,Broncos,2,16,609,399,65.5,...,56,12,4,0,0,0,0.000,480,349,131
2014-2015-Regular,AFC,65,IND,Indianapolis,Colts,3,16,669,410,61.3,...,52,11,5,0,0,0,0.000,458,367,91
2014-2015-Regular,AFC,57,CIN,Cincinnati,Bengals,4,16,501,321,64.1,...,39,10,5,0,0,0,0.000,359,346,13
2014-2015-Regular,AFC,59,PIT,Pittsburgh,Steelers,5,16,620,411,66.3,...,44,11,5,0,0,0,0.000,428,368,60
2014-2015-Regular,AFC,56,BAL,Baltimore,Ravens,6,16,559,341,61.0,...,44,10,6,0,0,0,0.000,403,302,101
2014-2015-Regular,AFC,75,SD,San Diego,Chargers,7,16,579,380,65.6,...,40,9,7,0,0,0,0.000,346,348,-2
2014-2015-Regular,AFC,73,KC,Kansas City,Chiefs,8,16,491,320,65.2,...,37,9,7,0,0,0,0.000,341,281,60
2014-2015-Regular,AFC,64,HOU,Houston,Texans,9,16,485,293,60.4,...,36,9,7,0,0,0,0.000,374,305,69


### Division (Each Season & Total)

In [79]:
division = "NFL_Standings/Division/Division_Standings_Combined.csv"
division = pd.read_csv(division)
division.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
division.set_index("Season")

,#Division Name,#Team ID,#Team Abbr.,#Team City,#Team Name,#Rank,#GamesPlayed,#PassAttempts,#PassCompletions,#PassPct,...,#TotalTD,#Wins,#Losses,#Ties,#OTWins,#OTLosses,#WinPct,#PointsFor,#PointsAgainst,#PointDifferential
Season,,,,,,,,,,,,,,,,,,,,,
2014-2015-Regular,AFC/AFC East,50,NE,New England,Patriots,1,16,627,392,62.5,...,51,12,4,0,0,0,0.000,461,313,148
2014-2015-Regular,AFC/AFC East,48,BUF,Buffalo,Bills,2,16,599,363,60.6,...,32,9,7,0,0,0,0.000,331,285,46
2014-2015-Regular,AFC/AFC East,49,MIA,Miami,Dolphins,3,16,601,394,65.6,...,40,8,8,0,0,0,0.000,384,373,11
2014-2015-Regular,AFC/AFC East,51,NYJ,New York,Jets,4,16,498,286,57.4,...,27,4,12,0,0,1,0.000,283,399,-116
2014-2015-Regular,AFC/AFC North,57,CIN,Cincinnati,Bengals,1,16,501,321,64.1,...,39,10,5,0,0,0,0.000,359,346,13
2014-2015-Regular,AFC/AFC North,59,PIT,Pittsburgh,Steelers,2,16,620,411,66.3,...,44,11,5,0,0,0,0.000,428,368,60
2014-2015-Regular,AFC/AFC North,56,BAL,Baltimore,Ravens,3,16,559,341,61.0,...,44,10,6,0,0,0,0.000,403,302,101
2014-2015-Regular,AFC/AFC North,58,CLE,Cleveland,Browns,4,16,512,276,53.9,...,30,7,9,0,0,0,0.000,298,337,-39
2014-2015-Regular,AFC/AFC South,65,IND,Indianapolis,Colts,1,16,669,410,61.3,...,52,11,5,0,0,0,0.000,458,367,91


In [ ]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# # for path, csv, subdirs, files in os.walk(root):
# #     for name in files:
# #         print os.path.join(path, name)
# # cumStats = glob.glob("Cumulative Stats/*.csv")

# #Append all files to have dates

# # for file in cumStats:
# #     convert_DF = cumStats.insert(0, "Season")
# #     convert_DF.insert(2, "Season", len(rows))

# #     convert_DF.to_csv(cumStats.csv)
# #Loop to Combine ALl DF
# cumStats_DF = []
# for filename in sorted(cumStats):
#     cumStats_DF.append(pd.read_csv(filename))
# fullStats_DF = pd.concat(cumStats_DF, join="inner")

# fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
# fullStats_DF
# # print(cumStats)

In [ ]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# for path, csv, subdirs, files in os.walk(root):
#     for name in files:
#         print os.path.join(path, name)
cumStats = glob.glob("Cumulative Stats/*.csv")

#Append all files to have dates

# for file in cumStats:
#     convert_DF = cumStats.insert(0, "Season")
#     convert_DF.insert(2, "Season", len(rows))

#     convert_DF.to_csv(cumStats.csv)
#Loop to Combine ALl DF
cumStats_DF = []
for filename in sorted(cumStats):
    cumStats_DF.append(pd.read_csv(filename))
fullStats_DF = pd.concat(cumStats_DF, join="inner")

fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
fullStats_DF
# print(cumStats)